In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import re
from os import path
from glob import glob
from tqdm import tqdm
import numpy as np
from matplotlib import cm
from pprint import pprint
from plotting_utils import get_legend_name, remove_outliers

In [ ]:
ROOT_PATH = '\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer'

In [ ]:
files = glob(path.join(ROOT_PATH, '*', 'MatchResult_*.csv'))


In [ ]:
StatisticsColumns = [
    'ReachedKnowledgeStep',
    'Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece']

In [ ]:
replacements = {
    'met': 'method',
    'pla': 'player',
    'eff': 'simpleEffect',
    'obj': 'objective',
    'sim': 'simulation',
    'rat': 'almostRatio',
    'pdep': 'playerDepth',
    'evo': 'simulation',
}

def parse_filename(data):
    result = dict()
    for i in range(0, len(data), 2):
        if data[i] in replacements.keys():
            data[i] = replacements[data[i]]

        result[data[i]] = data[i+1]

    return result

In [ ]:
dfs = []
for file_path in tqdm(files):
    # print(file_path)
    file_name = path.basename(file_path)

    _file_name = file_name.split('_')[1:-1]
    args = parse_filename(_file_name)

    df = pd.read_csv(file_path)

    for key, value in args.items():
        df[key] = value
    dfs.append(df)

original_df = pd.concat(dfs)

In [ ]:
original_df[original_df['method'] == 'sampling'].head()

In [ ]:
#
# original_df['Sum_Matched'] = original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']  + \
#     original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']
# original_df.loc[original_df['method'] == 'random', 'simulation'] = 0
#
# original_df['simpleEffect'] = original_df['simpleEffect'].replace(np.nan, 0)
# original_df['objective'] = original_df['objective'].replace(np.nan, '-')
#
# original_df.loc[original_df['method'] == 'random', 'playerDepth'] = '-'
# original_df.loc[original_df['method'] == 'sampling', 'playerDepth'] = '-'
# original_df.loc[original_df['method'] == 'ga', 'playerDepth'] = '-'
# original_df.loc[original_df['method'] == 'ga2', 'playerDepth'] = '-'
# original_df.loc[original_df['method'] == 'random', 'simulation'] = '-'
# original_df.loc[original_df['method'] == 'ga', 'simulation'] = '-'
# original_df.loc[original_df['method'] == 'ga2', 'simulation'] = '-'
# # original_df.loc[original_df['method'] == 'sampling', 'simulation'] = '-'
# original_df.loc[original_df['method'] == 'random', 'almostRatio'] = '-'
# original_df.loc[original_df['method'] == 'sampling', 'almostRatio'] = '-'
# original_df.loc[original_df['method'] == 'ga', 'almostRatio'] = '-'
# original_df.loc[original_df['objective'] == 'kp', 'almostRatio'] = '-'
# original_df.loc[original_df['objective'] == 'score', 'almostRatio'] = '-'

In [ ]:
# 완료하지 못했을 경우, 에피소드 최대길이로 설정
original_df['ReachedKnowledgeStep'] = original_df['ReachedKnowledgeStep'].replace(-1, 200)
original_df['Q1ReachedKnowledgeStep'] = original_df['Q1ReachedKnowledgeStep'].replace(-1, 200)
original_df['Q2ReachedKnowledgeStep'] = original_df['Q2ReachedKnowledgeStep'].replace(-1, 200)
original_df['Q3ReachedKnowledgeStep'] = original_df['Q3ReachedKnowledgeStep'].replace(-1, 200)


In [ ]:

original_df.loc[original_df['method'] == 'random', 'objective'] = '-'
original_df.loc[original_df['method'] == 'random', 'simulation'] = '-'
original_df.loc[original_df['method'] == 'sampling', 'objective'] = '-'

pd.set_option('display.max_rows', None)
original_df.groupby(['method', 'objective','simulation', 'player'])[
                                      'method',
                                      # 'Matched_HorizontalPiece',
                                      # 'Matched_VerticalPiece',
                                      # 'Matched_CrossPiece',
                                      # 'Matched_RocketPiece',
                                      # 'Matched_BombPiece',
                                      # 'Matched_RainbowPiece',
                                      # 'Q1ReachedKnowledgeStep',
                                      # 'Q2ReachedKnowledgeStep',
                                      # 'Q3ReachedKnowledgeStep',
                                      # 'ReachedKnowledgeStep',
                                      #'Sum_Matched'
                                      ].agg(
    {
        'method': ['count'],
        # 'Q1ReachedKnowledgeStep': ['mean'],
        # 'Q2ReachedKnowledgeStep': ['mean'],
        # 'Q3ReachedKnowledgeStep': ['mean'],
        # 'ReachedKnowledgeStep': ['mean'],
        # 'Matched_HorizontalPiece':['mean'],
        # 'Matched_VerticalPiece':['mean'],
        # 'Matched_CrossPiece':['mean'],
        # 'Matched_RocketPiece':['mean'],
        # 'Matched_BombPiece':['mean'],
        # 'Matched_RainbowPiece':['mean'],
        # 'Sum_Matched':['mean'],
     }
)

In [ ]:
original_df.columns

In [ ]:
vis_df = original_df.groupby(['method', 'objective','simulation', 'player'])[
                                        'StdGenerateRuntimes',
                                        'MeanGenerateRuntimes',
                                        'ReachedKnowledgeStep',
                                        'MeanGenerateScores',
                                        'StdGenerateScores',
                                      ].agg(
    {
        'StdGenerateRuntimes': ['mean'],
        'MeanGenerateRuntimes': ['mean'],
        'ReachedKnowledgeStep': ['mean', 'std'],
        'MeanGenerateScores': ['mean'],
        'StdGenerateScores': ['mean'],
    }).reset_index()
vis_df.columns = vis_df.columns.droplevel(level=1)  # Replace 0 with the level you want to remove


In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
vis_df.to_csv('simulation_result.csv')

In [ ]:
vis_df

In [ ]:
sns.lineplot(data=vis_df, x="MeanGenerateRuntimes", y="ReachedKnowledgeStep", hue="method")


In [ ]:
# sns.lineplot(data=vis_df, x="MeanGenerateRuntimes", y="MeanGenerateScores", hue="method", style="simulation")


### Full Result


### Depth 1 Result

In [ ]:
melted_df = match_result_noout.copy()

melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
pprint(set(melted_df['concat_method']))
# Display Target
melted_df = melted_df[melted_df['concat_method'].isin(['random', 'ga',
                                                       'sampling_20',
                                                       'mcts_knowledge_1.0_1000_1',
                                                       'mcts_score_1000_1'])]

plt.figure(figsize=(7, 5))
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'],
                    value_vars=[*StatisticsColumns], ignore_index=False)

melted_df['x'] = melted_df['variable']
melted_df['x'] = melted_df['x'].str.replace('ReachedKnowledgeStep', 'r_step')
melted_df['x'] = melted_df['x'].str.replace('Matched_HorizontalPiece', 'b0')
melted_df['x'] = melted_df['x'].str.replace('Matched_VerticalPiece', 'b1')
melted_df['x'] = melted_df['x'].str.replace('Matched_CrossPiece', 'b2')
melted_df['x'] = melted_df['x'].str.replace('Matched_RocketPiece', 'b3')
melted_df['x'] = melted_df['x'].str.replace('Matched_BombPiece', 'b4')
melted_df['x'] = melted_df['x'].str.replace('Matched_RainbowPiece', 'b5')

ax = sns.boxplot(x="x", y="value", hue="concat_method", data=melted_df, linewidth=0.8)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('matched_block.png')

In [ ]:
from statannot import add_stat_annotation
from itertools import combinations

In [ ]:
player_df = match_result_noout[match_result_noout['player'] == str(player_no)]
melted_df = player_df.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75'
}, inplace=True)

_columns = ['25', '50', '75', '100']

sns.set_palette('Set2')
# melted_df = melted_df.groupby(['player', 'method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'], value_vars=[*_columns], ignore_index=False)

melted_df = melted_df[melted_df['concat_method'].isin(display_graphs)]

plt.figure(figsize=(7, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

pairs = list()
for i in range(25, 101, 25):
    pairs.append(((str(i), 'mcts_knowledge_1.0_1000_1'), (str(i), 'mcts_score_1000_1')))
    pairs.append(((str(i), 'mcts_knowledge_1.0_1000_1'), (str(i), 'random')))
    pairs.append(((str(i), 'random'), (str(i), 'mcts_score_1000_1')))
    pairs.append(((str(i), 'random'), (str(i), 'sampling_20')))
    pairs.append(((str(i), 'mcts_score_1000_1'), (str(i), 'sampling_20')))
    pairs.append(((str(i), 'mcts_knowledge_1.0_1000_1'), (str(i), 'sampling_20')))

add_stat_annotation(ax, x="variable", y="value", hue="concat_method", data=melted_df,
                    box_pairs=pairs,
                    test='t-test_ind', text_format='star', loc='inside', verbose=1)

plt.legend(loc='upper left', bbox_to_anchor=(1.0, 0.5))
ax.set_title(f'Required Steps for Learning Skill (Player: {player_no})')
ax.set_xlabel('Progress (%)')
ax.set_ylabel('Step')
plt.tight_layout()
plt.savefig(f'./figures/learning_steps_p{player_no}', dpi=300)
    # plt.show()

In [ ]:

melted_df = match_result_noout.copy()

melted_df.rename(columns={
    'ReachedKnowledgeStep':'100',
    'Q1ReachedKnowledgeStep':'25',
    'Q2ReachedKnowledgeStep':'50',
    'Q3ReachedKnowledgeStep':'75'
}, inplace=True)

_columns = ['25', '50', '75', '100']

sns.set_palette('Set3')
# melted_df = melted_df.groupby(['player', 'method', 'objective', 'playerDepth', 'simulation']).sample(n=300, random_state=1)
melted_df['concat_method'] = melted_df.agg(lambda x: get_legend_name(x), axis=1)
melted_df = pd.melt(melted_df, id_vars=['EpisodeCount', 'objective', 'concat_method', 'simulation', 'player'], value_vars=[*_columns], ignore_index=False)

melted_df = melted_df[melted_df['concat_method'].isin(display_graphs)]


plt.figure(figsize=(7, 4))
plt.clf()
ax = sns.boxplot(x="variable", y="value", hue="concat_method", data=melted_df, dodge=True, linewidth=0.8)

pairs = list()
for i in range(25, 101, 25):
    pairs.append(((str(i), 'mcts_knowledge_1.0_1000_1'), (str(i), 'mcts_score_1000_1')))
    pairs.append(((str(i), 'mcts_knowledge_1.0_1000_1'), (str(i), 'random')))
    pairs.append(((str(i), 'random'), (str(i), 'mcts_score_1000_1')))
    pairs.append(((str(i), 'random'), (str(i), 'sampling_20')))
    pairs.append(((str(i), 'mcts_score_1000_1'), (str(i), 'sampling_20')))
    pairs.append(((str(i), 'mcts_knowledge_1.0_1000_1'), (str(i), 'sampling_20')))

add_stat_annotation(ax, x="variable", y="value", hue="concat_method", data=melted_df,
                    box_pairs=pairs,
                    test='t-test_ind', text_format='star', loc='inside', verbose=1)

plt.legend(loc='upper left', bbox_to_anchor=(1.0, 0.5))
ax.set_title(f'Required Steps for Learning Skill (Player: All)')
ax.set_xlabel('Progress (%)')
ax.set_ylabel('Step')
plt.savefig(f'./figures/learning_steps_pA', dpi=300)
# plt.show()

In [ ]:
original_df.columns

In [ ]:
original_df.groupby(['method'])['MeanGenerateRuntimes'].mean()